In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import time
import copy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline


In [9]:
map_am = gpd.read_file("Sidewalk_width_crossings.geojson")

# Objectives
objective1 = map_am['length']
objective2 = map_am['crossing']
objective3 = map_am['obstacle_free_width']

objectives = ('length', 'crossing')

In [10]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map_am, approach='primal')
nodes = G.nodes
edges = G.edges
# print(G)

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [11]:
print(nodes)

[(119998.5393221767, 485722.64175419795), (120007.44591842283, 485727.1636123485), (120015.18745424342, 485730.8926674651), (120061.19911276558, 485774.75863999536), (120051.28751834162, 485774.708468612), (120043.72763725856, 485768.3537648842), (120036.0837353018, 485761.99018608185), (120027.69801421888, 485756.6258637002), (120018.74070462542, 485752.3284842039), (120009.42954444634, 485748.80822725233), (119999.76652582387, 485746.4921293622), (120000.90194588286, 485844.03600847936), (120006.62757048407, 485848.54152873287), (120004.25910429093, 485824.64606569504), (120012.2328983722, 485829.886359004), (120012.60895610375, 485830.5125589196), (120036.9264611653, 485998.8336974648), (120028.54547317879, 485993.44651605055), (120019.19333015662, 485989.96658265253), (120009.68854709805, 485986.8936894776), (120000.16643816249, 485983.87730290537), (120002.96021868773, 485831.6557521264), (120009.91985786322, 486026.62236025586), (120001.66429379128, 486029.93018580374), (120005.3

In [12]:

# S = (122254.86602688645, 486129.80052856216) #very first node
# T = (122320.31466476223, 486327.5294561802)
S = (119998.5393221767, 485722.64175419795) #very first
T = (121544.5105401219, 486594.5264401745) #very last

In [13]:
def pareto_dominates(a, b):
    """Check if the vector in b Pareto dominates vector a.

    Note: The original code has been modified to work for our minimization problem.

    Args:
        a (ndarray): A numpy array.
        b (ndarray): A numpy array.

    Returns:
        bool: Whether vector b dominates vector a.
    """
    a = np.array(a)
    b = np.array(b)
    return np.all(a <= b) and np.any(a < b)


def p_prune(candidates):
    """Create a Pareto coverage set from a set of candidate points.

    References:
        .. [1] Roijers, D. M., & Whiteson, S. (2017). Multi-objective decision making. 34, 129–129.
            https://doi.org/10.2200/S00765ED1V01Y201704AIM034

    Args:
        candidates (Set[Tuple]): A set of vectors.

    Returns:
        Set[Tuple]: A Pareto coverage set.
    """
    pcs = set()
    while candidates:
        vector = candidates.pop()

        for alternative in candidates:
            if pareto_dominates(alternative, vector):
                vector = alternative

        to_remove = set(vector)
        for alternative in candidates:
            if pareto_dominates(vector, alternative):
                to_remove.add(alternative)

        candidates -= to_remove
        pcs.add(vector)
    return pcs


def pvi(G, T, objectives, max_iter=30):
    start = time.time()
    nd_vectors = [set([tuple(np.full(2, np.inf)) for _ in range(len(G.nodes))]) for _ in range(len(G.nodes))] # Initialisation of nodes
    j = 0

    for n, current_node in enumerate(G.nodes):
        if current_node == T:  # We've reached the terminal state
            nd_vectors[n] = set([(0, 0) for _ in G.nodes])
            break

    # nd_vectors_update = copy.deepcopy(nd_vectors)

    # for run in range(max_iter):  # We execute the algorithm for a number of iterations.
    #     print(f'Value Iteration number: {run}')

    # while '{(inf, inf)}' in str(nd_vectors_update):
    # converged = False
    # while not converged:  # Run until convergence
    #     converged = True
    while True:
        old_vectors = copy.deepcopy(nd_vectors)
        for n, current_node in enumerate(G.nodes):  # Loop over all states. Note: current_node is an object; n=number
            if current_node == T:
                continue

            for nk, neighbor in enumerate(G.nodes): #Note: neighbor is an object; k=number

                if neighbor not in G.neighbors(current_node):
                    continue

                edge = G[current_node][neighbor]
                edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties

                cost = []
                for i in objectives:
                  cost.append(edge_list[0][i])
                # print(f"cost:{cost}")
                cost = np.array(cost)
                results = nd_vectors[n].copy()

                for value_vec in nd_vectors[nk]:
                    # for value_vec in v_list:
                    # if value_vec != (np.inf, np.inf):
                    #     print("value vec:", value_vec)
                    results.add(tuple(cost+value_vec)) # The set of candidate vectors
                # print(f"results:{results}")

                results = p_prune(results)
                # print(f"results pruned:{results}")

                # if nd_vectors[n][nk] != results:
                #     converged = False
                nd_vectors[n] = results
                # print(f"nd_vectors:{nd_vectors}")


        j += 1
        # if has_converged(nd_vectors, old_vectors, 0.0001):
        #     break
        if nd_vectors == old_vectors:
            break
        # Check for convergence
        # if delta < 0.0001:
        #     break

        # if converged:
        #     break

        nd_vectors = copy.deepcopy(nd_vectors)  # Else perform a deep copy and go again.

    print(f'Iterations: {j}')
    end = time.time()
    elapsed_seconds = (end - start)
    print("Seconds elapsed: " + str(elapsed_seconds))

    return nd_vectors

# def has_converged(nd_vectors, old_vectors, epsilon):
#     for i in range(len(nd_vectors)):
#         # for j in range(len(nd_vectors[i])):
#         if not np.allclose(list(nd_vectors[i])[0], list(old_vectors[i])[0], atol=epsilon):
#             return False
#     return True

pvi_result = pvi(G, T, objectives)
print(pvi_result)


Iterations: 345
Seconds elapsed: 23097.188587903976
[{(2845.729999999991, 34.0), (3420.8699999999812, 21.0), (3119.6899999999823, 23.0), (2863.259999999991, 30.0), (3141.159999999982, 22.0), (2889.5999999999876, 27.0), (2875.1399999999876, 29.0), (2843.649999999991, 36.0), (2911.0699999999874, 26.0), (2855.2099999999914, 31.0), (2841.7099999999914, 37.0), (2881.549999999988, 28.0), (2961.8899999999862, 24.0), (2843.7899999999913, 35.0), (2848.799999999991, 32.0), (2940.4199999999864, 25.0)}, {(2833.659999999991, 36.0), (3109.6999999999825, 23.0), (2835.739999999991, 34.0), (2865.149999999988, 29.0), (2833.7999999999915, 35.0), (3131.1699999999823, 22.0), (2838.8099999999913, 32.0), (2831.7199999999916, 37.0), (2930.4299999999866, 25.0), (3410.8799999999815, 21.0), (2879.609999999988, 27.0), (2871.559999999988, 28.0), (2853.2699999999913, 30.0), (2845.2199999999916, 31.0), (2951.8999999999864, 24.0), (2901.0799999999876, 26.0)}, {(2825.0599999999913, 36.0), (2943.2999999999865, 24.0), (

In [14]:
from matplotlib import colors


def track_policy(G, S, T, vec, pvi_vec, tol=1e-8):
        """Track a policy from its return vector.

        Args:
            vec (array_like): The return vector to track.
            env (gym.Env): The environment to track the policy in.
            tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
        """
        target = np.array(vec)  # SET THE TARGET YOU WANT TO ACQUIRE. SO IN YOUR CASE THIS IS THE VECTOR BELONGING TO THE OPTIMAL PATH.
        path = []  # START THE “PATH” AT THE START NODE HERE.
        terminated = False  # WHETHER WE ARE AT THE TERMINAL NODE.
        total_rew = np.zeros(2)  # THE REWARD WE HAVE ACCUMULATED SO FAR IN THE PATH
        im_rew = np.zeros(2)

        for n, current_node in enumerate(G.nodes):
            if current_node == T:
                # total_rew += im_rew
                path.append(current_node)
                break
            else:  # WHILE WE DO NOT REACH THE GOAL NODE.
                closest_dist = np.inf  # THIS IS THE DISTANCE TO THE CLOSEST VALUE VECTOR WE CAN GET
                closest_action = 0  # THIS IS THE ACTION, I.E. FOR YOU THE NEIGHBOUR, THAT WE MUST GO WITH TO MOVE ALONG THE CORRECT PATH.
                found_action = False  # WHETHER WE THINK WE FOUND THE CORRECT NEIGHBOUR.
                new_target = target   # THE NEW TARGET

                for nk, action in enumerate(G.neighbors(current_node)):  # THIS CHECKS OVER THE POSSIBLE OPTIONS YOU CAN DO. FOR YOU THIS WOULD BE THE NEIGHBOURS OF YOUR CURRENT STATE.

                    # CHECK WHAT COST IS ASSOCIATED WITH THE EDGE BETWEEN YOUR CURRENT NODE AND THE NEIGHBOUR.
                    edge = G[current_node][action]
                    edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties

                    im_rew = np.array([edge_list[0]['length'], edge_list[0]['crossing']])
                    # print(f"im_rew:{im_rew}")
                    non_dominated_set = pvi_vec[n]  # GET THE NON DOMINATED SET FROM PVI AT THIS NODE-NEIGHBOUR

                    for q in non_dominated_set:  # CHECK FOR EVERY VECTOR IN THIS SET
                        q = np.array(q)
                        dist = np.sum(np.abs(q + im_rew - target))  # CHECK THE DISTANCE BETWEEN YOUR TARGET (SO WHAT YOU STILL NEED TO GET) AND THE VALUE VECTOR
                        if dist < closest_dist:  # IF THE DISTANCE IS BETTER THAN WHAT YOU GOT SO FAR.
                            closest_dist = dist  # SET THIS DISTANCE AS THE BEST DISTANCE.
                            closest_action = action  # TAKE THE ACTION.
                            new_target = q  # UPDATE THE TARGET (I.E. REMOVE THE IMMEDIATE COST)
                            # print(new_target)
                            if dist < tol:  # IF THE DISTANCE IS GOOD ENOUGH.
                                found_action = True  # JUST SAY YOU FOUND THE CORRECT NEIGHBOUR
                                path.append(current_node)
                                # print("current_node", current_node)
                                break  # AND STOP THE LOOP


                    if found_action:
                        # path.append(closest_action)
                        break

                current_node = closest_action  # TAKE THE EDGE THAT YOU THINK IS BEST
                # edge2 = G[closest_action][current_node]
                total_rew = im_rew # UPDATE THE REWARD YOU ACQUIRED from tracking the policy and should equal the target vector in the end
                target = new_target  # UPDATE THE TARGET
                # if current_node not in path:
                #     path.append(current_node)


        return total_rew, path  # YOU HAVE TO RETURN THE TRAVERSED PATH HERE AS WELL


# res, pathh = track_policy(G, S, T, (284.05,4.), pvi_result)
# print(res, pathh)

In [15]:
def track_policy(G, S, T, target, pvi_vec, tol=1e-3):

        terminated = False
        truncated = False
        reward = np.zeros(2)
        path = []

        while not (terminated or truncated):
            closest_dist = np.inf
            closest_action = 0
            found_action = False
            new_target = target

            for n, current_node in enumerate(G.nodes):
                if current_node == T:
                    terminated = True
                    truncated = True
                else:
                    for nk, neighbor in enumerate(G.neighbors(current_node)):
                        im_rew = [n, nk]
                        # print(f"im_rew:{im_rew}")
                        non_dominated_set = pvi_vec[n]
                        # print(f"non_dominated set:{non_dominated_set}")

                        for q in non_dominated_set:
                            q = np.array(q)
                            # print(f"q:{q}")
                            dist = np.sum(np.abs(q + im_rew - target))
                            # print(f"distance:{dist}")
                            if dist < closest_dist:
                                closest_dist = dist
                                closest_action = nk
                                new_target = q

                                if dist < tol:
                                    found_action = True
                                    break

                        if found_action:
                            break

                #closest action = edge you want to take
                #step = go to neighbor node
                # closest_action = nk
                reward = im_rew
                target = new_target
                if current_node not in path:
                    path.append(current_node)

        return reward, path

# res, path = track_policy(G, S, T, (284.05,4.), pvi_result)
# print(path)

In [16]:
#ALTERNATIVE CODE I TRIED
def track_policy(G, S, T, vec, pvi_vec, tol=1e-3):
        """Track a policy from its return vector.

        Args:
            vec (array_like): The return vector to track.
            env (gym.Env): The environment to track the policy in.
            tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
        """
        target = np.array(vec)  # SET THE TARGET YOU WANT TO ACQUIRE. SO IN YOUR CASE THIS IS THE VECTOR BELONGING TO THE OPTIMAL PATH.
        path = [S]  # START THE “PATH” AT THE START NODE HERE.
        terminated = False  # WHETHER WE ARE AT THE TERMINAL NODE.
        # truncated = False  # YOU CAN IGNORE THIS
        total_rew = np.zeros(2)  # THE REWARD WE HAVE ACCUMULATED SO FAR IN THE PATH
        # current_gamma = 1.0  # YOU CAN IGNORE THIS.

        for n, current_node in enumerate(G.nodes):
            if current_node == T:
                break
            else:  # WHILE WE DO NOT REACH THE GOAL NODE.
                closest_dist = np.inf  # THIS IS THE DISTANCE TO THE CLOSEST VALUE VECTOR WE CAN GET
                closest_action = 0  # THIS IS THE ACTION, I.E. FOR YOU THE NEIGHBOUR, THAT WE MUST GO WITH TO MOVE ALONG THE CORRECT PATH.
                found_action = False  # WHETHER WE THINK WE FOUND THE CORRECT NEIGHBOUR.
                new_target = target   # THE NEW TARGET

                for nk, action in enumerate(G.neighbors(current_node)):  # THIS CHECKS OVER THE POSSIBLE OPTIONS YOU CAN DO. FOR YOU THIS WOULD BE THE NEIGHBOURS OF YOUR CURRENT STATE.
                    # im_rew = [n, nk]  # CHECK WHAT COST IS ASSOCIATED WITH THE EDGE BETWEEN YOUR CURRENT NODE AND THE NEIGHBOUR.
                    edge = G[current_node][action]
                    edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties

                    im_rew = np.array([edge_list[0]['length'], edge_list[0]['crossing']])
                    print(f"im_rew:{im_rew}")
                    non_dominated_set = pvi_vec[n]  # GET THE NON DOMINATED SET FROM PVI AT THIS NODE-NEIGHBOUR
                    print(f"non-dominated:{non_dominated_set}")

                    for q in non_dominated_set:  # CHECK FOR EVERY VECTOR IN THIS SET
                        q = np.array(q)
                        dist = np.sum(np.abs(q + im_rew - target))  # CHECK THE DISTANCE BETWEEN YOUR TARGET (SO WHAT YOU STILL NEED TO GET) AND THE VALUE VECTOR
                        if dist < closest_dist:  # IF THE DISTANCE IS BETTER THAN WHAT YOU GOT SO FAR.
                            closest_dist = dist  # SET THIS DISTANCE AS THE BEST DISTANCE.
                            closest_action = action  # TAKE THE ACTION.
                            new_target = q  # UPDATE THE TARGET (I.E. REMOVE THE IMMEDIATE COST)

                            if dist < tol:  # IF THE DISTANCE IS GOOD ENOUGH.
                                found_action = True  # JUST SAY YOU FOUND THE CORRECT NEIGHBOUR
                                break  # AND STOP THE LOOP

                    if found_action:
                        break

                current_node = closest_action  # TAKE THE EDGE THAT YOU THINK IS BEST
                path.append(current_node)
                total_rew += im_rew  # UPDATE THE REWARD YOU ACQUIRED.
                target = new_target  # UPDATE THE TARGET

        return total_rew, path  # YOU HAVE TO RETURN THE TRAVERSED PATH HERE AS WELL

tt = (284.05,8.)
# tt = (284.05,  14.)
# ress, pathh = track_policy(G, S, T, tt, pvi_result)
# print(pathh)

In [ ]:
# def track_policy():
#         """Track a policy from its return vector.
#
#         Args:
#             vec (array_like): The return vector to track.
#             G: Graph
#             tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
#         """
#         # target = np.array(vec)
#         # state, _ = env.reset()
#         terminated = False
#         truncated = False
#         total_rew = np.zeros(self.num_objectives)
#
#         while not (terminated or truncated):
#             # state = np.ravel_multi_index(state, self.env_shape)
#             closest_dist = np.inf
#             closest_action = 0
#             found_action = False
#             new_target = target
#
#             for n, current_node in enumerate(G.nodes):
#                 for nk, neighbor in enumerate(G.neighbors(current_node)):
#                     im_rew = self.avg_reward[n, nk]
#                     non_dominated_set = self.non_dominated[n][nk]
#
#                     for q in non_dominated_set:
#                         q = np.array(q)
#                         dist = np.sum(np.abs(q + im_rew - t))
#                         if dist < closest_dist:
#                             closest_dist = dist
#                             closest_action = nk
#                             new_target = q
#
#                             if dist < tol:
#                                 found_action = True
#                                 break
#
#                     if found_action:
#                         break
#
#                 #closest action = edge you want to take
#                 #step = go to neighbor node
#                 state, reward, terminated, truncated, _ = G.nodes(closest_action) #go to the node that it's at the end of the edge
#                 total_rew += reward
#                 target = new_target
#
#         return total_rew
